## Описание и проверка гипотез

### Импорты

In [5]:
import os
import kagglehub
import cv2
import src.image_classification.features as feat
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from scipy.stats import mannwhitneyu, spearmanr, kruskal, levene, ttest_ind, pearsonr, chi2_contingency, f_oneway

### Основной путь и классы, с которыми будет идти работа

In [6]:
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
classes = os.listdir(os.path.join(path, "garbage-dataset"))

### Гипотеза №1 О наличие картона на картинке можно судить по скосу желтого цвета.

В данной гипотезе мы предполагаем, что цветовая характеристика изображения, а именно наличие желтого цвета, может служить индикатором присутствия картона на картинке. Исходя из наблюдений, мы предполагаем, что изображения, содержащие картон, будут иметь более выраженный желтый оттенок по сравнению с изображениями, не содержащими этот материал.


- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Этот метод будет использован для сравнения средней доли желтого цвета между двумя группами. Мы проверим нулевую гипотезу о том, что средние значения доли желтого цвета в обеих группах не отличаются.

    - **ANOVA (дисперсионный анализ)**: Если у нас есть более двух групп (например, различные виды картона и другие материалы), ANOVA поможет определить, есть ли значимые различия в количестве желтого цвета между всеми группами.

    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае картон и другие классы).

In [7]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None: 
            images.append(image)
    return images

yellow_feature = feat.AmountOfYellow()

cardboard_images = load_images_from_folder(cardboard_path)
cardboard_yellow_amounts = [yellow_feature(img) for img in cardboard_images]

other_yellow_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_yellow_amounts.extend([yellow_feature(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(cardboard_yellow_amounts, other_yellow_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_yellow_amounts, other_yellow_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(cardboard_yellow_amounts, other_yellow_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие картона связано c желтым цветом.")
else:
    print("Итог: гипотеза не подтверждена: наличие картона не связано c желтым цветом.")

t-test: Statistic = 3.683066343249196, p-value = 0.00023621963453461276
ANOVA: Statistic = 16.865194578791744, p-value = 4.0292196417557166e-05
Mann-Whitney: Statistic = 22080315.5, p-value = 1.0126318223771204e-146
Итог: гипотеза подтверждена: наличие картона связано c желтым цветом.


### Гипотеза №2 О наличие металла на картинке можно судить по скосу серого (серебряного) цвета.

В данной гипотезе мы предполагаем, что цветовая характеристика изображения, а именно наличие серого или серебряного цвета, может служить индикатором присутствия металла на картинке. Мы предполагаем, что изображения, содержащие металл, будут иметь более выраженный серый оттенок по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Этот метод будет использован для сравнения средней доли серебряного цвета между двумя группами. Мы проверим нулевую гипотезу о том, что средние значения доли серебряного цвета в обеих группах не отличаются.

    - **ANOVA (дисперсионный анализ)**: Если у нас есть более двух групп (например, различные виды металлов и другие материалы), ANOVA поможет определить, есть ли значимые различия в количестве серебряного цвета между всеми группами. 

    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае металл и другие классы).

In [8]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

silver_feature = feat.AmountOfSilver()

metal_images = load_images_from_folder(metal_path)
metal_silver_amounts = [silver_feature(img) for img in metal_images]

other_silver_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_silver_amounts.extend([silver_feature(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_silver_amounts, other_silver_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(metal_silver_amounts, other_silver_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(metal_silver_amounts, other_silver_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие металла связано c серебряным цветом.")
else:
    print("Итог: гипотеза не подтверждена: наличие металла не связано c серебряным цветом.")

t-test: Statistic = -2.284424004913432, p-value = 0.022401779559765453
ANOVA: Statistic = 0.5733189930562239, p-value = 0.448952071385762
Mann-Whitney: Statistic = 10996543.5, p-value = 5.262970510833023e-16
Итог: гипотеза не подтверждена: наличие металла не связано c серебряным цветом.


### Гипотеза №3 На картинках с картоном присутствует множество параллельных линий.

На картинках с картоном присутствует множество параллельных линий, что может быть связано с характерными текстурами и структурой материала. Данная гипотеза предполагает, что изображения с картоном содержат больше параллельных линий по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: тот метод позволяет сравнить среднюю долю параллельных линий в картинках с картоном и других классах. Мы рассчитываем t-статистику и p-значение, чтобы определить, есть ли статистически значимые различия между двумя группами.

    - **ANOVA (дисперсионный анализ)**: Данный подход позволяет проверить, существуют ли значимые различия между несколькими группами (в данном случае между картоном и другими классами) по количеству параллельных линий. ANOVA помогает определить, влияет ли класс изображения на количество параллельных линий.

    - **Критерий хи-квадрат**: Этот тест позволяет проверить гипотезу о различии долей изображений с высокими количествами параллельных линий между картоном и другими классами. Мы анализируем, сколько изображений из каждой группы попадает в категорию с высоким количеством параллельных линий и сравниваем эти данные.

In [9]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

# Load images
cardboard_images = load_images_from_folder(cardboard_path)

# Create an instance of the AmountOfParallelLines feature extractor
parallel_lines_feature_extractor = feat.AmountOfParallelLines()

# Calculate features for cardboard images
cardboard_features = [parallel_lines_feature_extractor(img) for img in cardboard_images]

other_features = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_features.extend([parallel_lines_feature_extractor(img) for img in other_images])

# Hypothesis testing
# 1. Student's t-test
t_stat, t_pvalue = ttest_ind(cardboard_features, other_features, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_features, other_features)

# 3. Chi-square test
threshold = np.percentile(cardboard_features, 75) 
cardboard_high = sum(x >= threshold for x in cardboard_features)
cardboard_low = len(cardboard_features) - cardboard_high
other_high = sum(x >= threshold for x in other_features)
other_low = len(other_features) - other_high
chi2_stat, chi2_pvalue, _, _ = chi2_contingency([[cardboard_high, cardboard_low], [other_high, other_low]])

# Collect results
results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Chi-square": {"statistic": chi2_stat, "p-value": chi2_pvalue},
}

# Print results
for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

# Final hypothesis check
alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: изображения c картоном имеют больше параллельных линий.")
else:
    print("Итог: гипотеза не подтверждена: изображения c картоном не имеют значительно больше параллельных линий.")

t-test: Statistic = -11.379057124078555, p-value = 1.9776123451846287e-29
ANOVA: Statistic = 52.405427227996164, p-value = 4.680228101392663e-13
Chi-square: Statistic = 19.34257889433447, p-value = 1.0924297310604447e-05
Итог: гипотеза подтверждена: изображения c картоном имеют больше параллельных линий.


### Гипотеза №5 Металлические поверхности часто имеют высокую отражающую способность. 

Металлические поверхности, как правило, обладают высокой отражающей способностью, что делает их уникальными в сравнении с другими материалами. Высокая отражающая способность металла может влиять на визуальное восприятие объектов. В данном эксперименте мы стремимся проверить, действительно ли металлические поверхности обладают более высоким уровнем отражений по сравнению с другими классами материалов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test):**  Этот метод позволяет сравнить среднюю отражающую способность изображений металла и других классов. Мы будем использовать его для определения, существуют ли статистически значимые различия между этими двумя группами.

    - **Корреляция Пирсона:** С помощью этого теста мы проверим наличие линейной связи между количеством отражений и принадлежностью к классу "металл". Это поможет нам понять, насколько сильно отражающая способность связана с материалом.

    - **Критерий хи-квадрат:** Этот тест будет использован для проверки отличий доли изображений с высоким уровнем отражений между металлом и другими классами. Мы разделим изображения на две группы: с высоким уровнем отражений и с низким, и проверим, есть ли статистически значимые различия между ними.

In [10]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

reflection_calculator = feat.AmountOfReflections()

metal_images = load_images_from_folder(metal_path)
metal_reflections = [reflection_calculator(img) for img in metal_images]

other_reflections = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_reflections.extend([reflection_calculator(img) for img in other_images])

# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_reflections, other_reflections, equal_var=False)

# 2. Корреляция Пирсона
le = LabelEncoder()
all_reflections = metal_reflections + other_reflections
all_labels = ['metal'] * len(metal_reflections) + ['other'] * len(other_reflections)
numeric_labels = le.fit_transform(all_labels)

correlation, p_value_corr = pearsonr(all_reflections, numeric_labels)

# 3. Тест хи-квадрат
reflection_binary = [1 if reflection >= 100 else 0 for reflection in all_reflections]
contingency_table = pd.crosstab(reflection_binary, numeric_labels)
chi2_stat, p_value_chi2, _, _ = chi2_contingency(contingency_table)

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "Pearson Correlation": {"statistic": correlation, "p-value": p_value_corr},
    "Chi-square": {"statistic": chi2_stat, "p-value": p_value_chi2},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие металла связано с высоким уровнем отражений.")
else:
    print("Итог: гипотеза не подтверждена: наличие металла не связано с высоким уровнем отражений.")

t-test: Statistic = -2.5612454595984153, p-value = 0.010464870898548239
Pearson Correlation: Statistic = 0.005891723334828736, p-value = 0.407558471373331
Chi-square: Statistic = 35.91242941824723, p-value = 2.0638790312840235e-09
Итог: гипотеза не подтверждена: наличие металла не связано с высоким уровнем отражений.
